# Experiment 9: Recovery Dynamics with Asymmetric Mechanisms

**Phase 4 - Testing Biologically-Realistic Hysteresis**

## Background

Experiment 8 revealed that symmetric dynamics produce symmetric outcomes: the tip/recovery ratio remained ~1.0 even under extreme network fragmentation. This contradicts industry research showing strong hysteresis in real ecosystems.

**Industry Research Shows** real ecosystem asymmetry arises from:
1. **Biological timescales**: Trees grow in decades, burn in hours
2. **State-dependent feedbacks**: Fire begets fire, drought begets drought
3. **Climate trends**: Warming is unidirectional, not symmetric noise
4. **Biodiversity loss**: Recovery capacity degrades after tipping

## Methodology Update (Option D)

Initial runs showed 0% recovery for ALL conditions (including symmetric baseline) due to:
- Barrier height (0.5) too strong relative to recovery noise (0.02)
- Deterministic restoring force >> noise amplitude

**Option D Parameters** fix this by:
- Reducing barrier heights: 0.2 (tip) to 0.4-0.6 (recovery)
- Increasing recovery noise: 0.04 (from 0.02)
- Maintaining 2-3x asymmetry ratio

## This Experiment Tests

Four asymmetric mechanisms and their combinations:

| Mechanism | Implementation | Expected Effect |
|-----------|----------------|----------------|
| Barrier asymmetry | `barrier_recovery = 2-3x barrier_tip` | Moderate hysteresis |
| Coupling degradation | Tipped cells provide 20-30% support | Feedback loops |
| Combined effects | Both mechanisms | Strong hysteresis |

**Success Criterion**: Full realistic condition produces recovery fraction < 20% while symmetric baseline shows ~40-50%

## 1. Setup and Imports

In [ ]:
import sys
sys.path.insert(0, '/opt/research-local/src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from pathlib import Path

# Core energy-constrained module
from energy_constrained import (
    EnergyConstrainedNetwork,
    EnergyConstrainedCusp,
    AsymmetricBarrierCusp,
    HistoryDependentCusp,
    GradientDrivenCoupling,
    StateDependentCoupling,
    TrendForcingCoupling,
    run_two_phase_experiment,
    run_two_phase_ensemble,
    aggregate_two_phase_results,
    EnergyAnalyzer,
    get_dask_client
)

print("Imports successful!")
print(f"NumPy version: {np.__version__}")

In [ ]:
# Connect to Dask cluster
client = get_dask_client()
print(f"Connected to: {client.scheduler_info()['address']}")
print(f"Workers: {len(client.scheduler_info()['workers'])}")
print(f"Total threads: {sum(w['nthreads'] for w in client.scheduler_info()['workers'].values())}")

## 2. Load Amazon Moisture Recycling Data

In [ ]:
# Load Amazon moisture recycling data (same approach as notebook 06)
from netCDF4 import Dataset

DATA_PATH = Path('/opt/research-local/data/amazon/amazon_adaptation_model/average_network/era5_new_network_data')

def load_amazon_data(year=2003, months=[7, 8, 9]):
    """Load and average Amazon moisture recycling data for specified months."""
    all_rain = []
    all_evap = []
    all_network = []
    
    for month in months:
        file_path = DATA_PATH / f'1deg_{year}_{month:02d}.nc'
        if file_path.exists():
            with Dataset(file_path, 'r') as ds:
                all_rain.append(ds.variables['rain'][:])
                all_evap.append(ds.variables['evap'][:])
                all_network.append(ds.variables['network'][:])
    
    return {
        'rain': np.mean(all_rain, axis=0),
        'evap': np.mean(all_evap, axis=0),
        'network': np.mean(all_network, axis=0),
        'n_cells': len(all_rain[0])
    }

# Load 2003 (normal year) data
amazon_data = load_amazon_data(year=2003)
print(f"Loaded Amazon data: {amazon_data['n_cells']} cells")
print(f"Network shape: {amazon_data['network'].shape}")
print(f"Rain range: {amazon_data['rain'].min():.1f} - {amazon_data['rain'].max():.1f} mm")

## 3. Network Creation Functions with Asymmetric Elements

In [ ]:
def create_symmetric_network(data, n_cells=50, min_flow=1.0, seed=42):
    """
    Create network with SYMMETRIC barriers (baseline - like Exp 8).
    Option D: barrier_height=0.2 (reduced from 0.5 to allow recovery)
    """
    np.random.seed(seed)
    
    network_matrix = data['network']
    rain = data['rain']
    evap = data['evap']
    
    # Select top cells by total connectivity
    total_flow = network_matrix.sum(axis=0) + network_matrix.sum(axis=1)
    top_indices = np.argsort(total_flow)[-n_cells:]
    
    # Create network
    net = EnergyConstrainedNetwork()
    
    # Add elements with SYMMETRIC barriers (Option D: lower barriers)
    for i, idx in enumerate(top_indices):
        element = EnergyConstrainedCusp(
            a=-1.0, b=1.0, c=0.0, x_0=0.0,
            barrier_height=0.2,  # OPTION D: Reduced from 0.5
            dissipation_rate=0.1
        )
        net.add_element(f'cell_{i}', element)
    
    # Add couplings based on moisture flow
    n_edges = 0
    for i, idx_i in enumerate(top_indices):
        for j, idx_j in enumerate(top_indices):
            if i != j:
                flow = network_matrix[idx_i, idx_j]
                if flow > min_flow:
                    coupling = GradientDrivenCoupling(
                        conductivity=flow / 100.0,
                        state_coupling=0.1
                    )
                    net.add_coupling(f'cell_{i}', f'cell_{j}', coupling)
                    n_edges += 1
    
    print(f"Created symmetric network: {net.n_elements} nodes, {n_edges} edges (barrier=0.2)")
    return net, top_indices


def create_asymmetric_barrier_network(data, n_cells=50, min_flow=1.0, 
                                       barrier_tip=0.2, barrier_recovery=0.5, 
                                       seed=42):
    """
    Create network with ASYMMETRIC barriers.
    Option D: barrier_tip=0.2, barrier_recovery=0.4-0.6 (2-3x ratio)
    """
    np.random.seed(seed)
    
    network_matrix = data['network']
    
    # Select top cells by total connectivity
    total_flow = network_matrix.sum(axis=0) + network_matrix.sum(axis=1)
    top_indices = np.argsort(total_flow)[-n_cells:]
    
    net = EnergyConstrainedNetwork()
    
    # Add elements with ASYMMETRIC barriers
    for i, idx in enumerate(top_indices):
        element = AsymmetricBarrierCusp(
            a=-1.0, b=1.0, c=0.0, x_0=0.0,
            barrier_tip=barrier_tip,
            barrier_recovery=barrier_recovery,
            dissipation_rate=0.1
        )
        net.add_element(f'cell_{i}', element)
    
    # Add couplings
    n_edges = 0
    for i, idx_i in enumerate(top_indices):
        for j, idx_j in enumerate(top_indices):
            if i != j:
                flow = network_matrix[idx_i, idx_j]
                if flow > min_flow:
                    coupling = GradientDrivenCoupling(
                        conductivity=flow / 100.0,
                        state_coupling=0.1
                    )
                    net.add_coupling(f'cell_{i}', f'cell_{j}', coupling)
                    n_edges += 1
    
    print(f"Created asymmetric barrier network: {net.n_elements} nodes, {n_edges} edges")
    print(f"  barrier_tip={barrier_tip}, barrier_recovery={barrier_recovery}, ratio={barrier_recovery/barrier_tip:.1f}x")
    return net, top_indices


def create_state_dependent_network(data, n_cells=50, min_flow=1.0,
                                    degradation_factor=0.3, seed=42):
    """
    Create network with STATE-DEPENDENT couplings.
    Tipped cells provide reduced support (degradation_factor) to neighbors.
    Option D: Uses lower barrier (0.2)
    """
    np.random.seed(seed)
    
    network_matrix = data['network']
    
    # Select top cells by total connectivity
    total_flow = network_matrix.sum(axis=0) + network_matrix.sum(axis=1)
    top_indices = np.argsort(total_flow)[-n_cells:]
    
    net = EnergyConstrainedNetwork()
    
    # Add symmetric elements with Option D barrier
    for i, idx in enumerate(top_indices):
        element = EnergyConstrainedCusp(
            a=-1.0, b=1.0, c=0.0, x_0=0.0,
            barrier_height=0.2,  # OPTION D: Reduced from 0.5
            dissipation_rate=0.1
        )
        net.add_element(f'cell_{i}', element)
    
    # Add STATE-DEPENDENT couplings
    n_edges = 0
    for i, idx_i in enumerate(top_indices):
        for j, idx_j in enumerate(top_indices):
            if i != j:
                flow = network_matrix[idx_i, idx_j]
                if flow > min_flow:
                    coupling = StateDependentCoupling(
                        base_conductivity=flow / 100.0,
                        degradation_factor=degradation_factor
                    )
                    net.add_coupling(f'cell_{i}', f'cell_{j}', coupling)
                    n_edges += 1
    
    print(f"Created state-dependent network: {net.n_elements} nodes, {n_edges} edges")
    print(f"  degradation_factor={degradation_factor}, barrier=0.2")
    return net, top_indices


def create_full_asymmetric_network(data, n_cells=50, min_flow=1.0,
                                    barrier_tip=0.2, barrier_recovery=0.5,
                                    degradation_factor=0.3, seed=42):
    """
    Create network with BOTH asymmetric barriers AND state-dependent coupling.
    This is the 'full realistic' condition.
    Option D: barrier_tip=0.2, barrier_recovery=0.4-0.6
    """
    np.random.seed(seed)
    
    network_matrix = data['network']
    
    # Select top cells by total connectivity
    total_flow = network_matrix.sum(axis=0) + network_matrix.sum(axis=1)
    top_indices = np.argsort(total_flow)[-n_cells:]
    
    net = EnergyConstrainedNetwork()
    
    # Add ASYMMETRIC elements
    for i, idx in enumerate(top_indices):
        element = AsymmetricBarrierCusp(
            a=-1.0, b=1.0, c=0.0, x_0=0.0,
            barrier_tip=barrier_tip,
            barrier_recovery=barrier_recovery,
            dissipation_rate=0.1
        )
        net.add_element(f'cell_{i}', element)
    
    # Add STATE-DEPENDENT couplings
    n_edges = 0
    for i, idx_i in enumerate(top_indices):
        for j, idx_j in enumerate(top_indices):
            if i != j:
                flow = network_matrix[idx_i, idx_j]
                if flow > min_flow:
                    coupling = StateDependentCoupling(
                        base_conductivity=flow / 100.0,
                        degradation_factor=degradation_factor
                    )
                    net.add_coupling(f'cell_{i}', f'cell_{j}', coupling)
                    n_edges += 1
    
    print(f"Created full asymmetric network: {net.n_elements} nodes, {n_edges} edges")
    print(f"  barrier_tip={barrier_tip}, barrier_recovery={barrier_recovery}, ratio={barrier_recovery/barrier_tip:.1f}x")
    print(f"  degradation_factor={degradation_factor}")
    return net, top_indices


print("Network creation functions defined (Option D parameters).")

## 4. Test Single Two-Phase Experiment

In [ ]:
# Create symmetric baseline network for comparison
# UPDATED: Using Option D parameters - lower barriers, higher recovery noise
network_sym, selected_cells = create_symmetric_network(amazon_data, n_cells=50, seed=42)
print(f"Symmetric network: {network_sym.n_elements} elements, {network_sym.number_of_edges()} edges")

# Run single two-phase experiment
# Option D: recovery_sigma=0.04 (up from 0.02) to allow recovery
result_sym = run_two_phase_experiment(
    network=network_sym,
    cascade_duration=200,
    recovery_duration=800,
    dt=0.5,
    cascade_sigma=0.06,
    cascade_alpha=1.5,
    recovery_sigma=0.04,  # INCREASED from 0.02 to allow recovery
    recovery_alpha=2.0,
    seed=123
)

print("\n=== Symmetric Baseline Results ===")
print(f"Tipped at cascade end: {result_sym.metrics['pct_tipped_at_cascade_end']:.1f}%")
print(f"Recovery fraction: {result_sym.metrics['recovery_fraction']:.2f}")
print(f"Mean recovery time: {result_sym.metrics['mean_recovery_time']:.1f}")
print(f"Permanent tips: {result_sym.metrics['n_permanent_tips']}")
print(f"Final tipped: {result_sym.metrics['final_pct_tipped']:.1f}%")

In [ ]:
# Create full asymmetric network with Option D parameters
# barrier_tip=0.2, barrier_recovery=0.5 (2.5x ratio)
network_asym, _ = create_full_asymmetric_network(
    amazon_data, n_cells=50, 
    barrier_tip=0.2,        # REDUCED from 0.5 to allow noise-driven transitions
    barrier_recovery=0.5,   # 2.5x harder to recover (was 1.5 with old barrier_tip=0.5)
    degradation_factor=0.3,  # Tipped cells provide 30% support
    seed=42
)
print(f"Asymmetric network: {network_asym.n_elements} elements")

# Run single experiment with Option D recovery noise
result_asym = run_two_phase_experiment(
    network=network_asym,
    cascade_duration=200,
    recovery_duration=800,
    dt=0.5,
    cascade_sigma=0.06,
    cascade_alpha=1.5,
    recovery_sigma=0.04,    # INCREASED from 0.02
    recovery_alpha=2.0,
    seed=123
)

print("\n=== Full Asymmetric Results (Option D) ===")
print(f"Parameters: barrier_tip=0.2, barrier_recovery=0.5, recovery_sigma=0.04")
print(f"Tipped at cascade end: {result_asym.metrics['pct_tipped_at_cascade_end']:.1f}%")
print(f"Recovery fraction: {result_asym.metrics['recovery_fraction']:.2f}")
print(f"Mean recovery time: {result_asym.metrics['mean_recovery_time']:.1f}")
print(f"Permanent tips: {result_asym.metrics['n_permanent_tips']}")
print(f"Final tipped: {result_asym.metrics['final_pct_tipped']:.1f}%")

In [ ]:
# Plot comparison of trajectories
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Symmetric: % tipped over time
ax = axes[0, 0]
pct_tipped_sym = np.mean(result_sym.x_full > 0, axis=1) * 100
ax.plot(result_sym.t_full, pct_tipped_sym, 'b-', lw=2)
ax.axvline(200, color='r', linestyle='--', label='Recovery phase starts')
ax.set_xlabel('Time')
ax.set_ylabel('% Tipped')
ax.set_title('Symmetric Baseline: % Tipped Over Time')
ax.legend()
ax.grid(True, alpha=0.3)

# Asymmetric: % tipped over time
ax = axes[0, 1]
pct_tipped_asym = np.mean(result_asym.x_full > 0, axis=1) * 100
ax.plot(result_asym.t_full, pct_tipped_asym, 'r-', lw=2)
ax.axvline(200, color='r', linestyle='--', label='Recovery phase starts')
ax.set_xlabel('Time')
ax.set_ylabel('% Tipped')
ax.set_title('Full Asymmetric: % Tipped Over Time')
ax.legend()
ax.grid(True, alpha=0.3)

# Compare recovery curves
ax = axes[1, 0]
ax.plot(result_sym.t_full, pct_tipped_sym, 'b-', lw=2, label='Symmetric')
ax.plot(result_asym.t_full, pct_tipped_asym, 'r-', lw=2, label='Asymmetric')
ax.axvline(200, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Time')
ax.set_ylabel('% Tipped')
ax.set_title('Comparison: Symmetric vs Asymmetric Recovery')
ax.legend()
ax.grid(True, alpha=0.3)

# Summary metrics bar chart
ax = axes[1, 1]
metrics = ['Cascade\n% Tipped', 'Recovery\nFraction', 'Permanent\nTips']
sym_vals = [
    result_sym.metrics['pct_tipped_at_cascade_end'],
    result_sym.metrics['recovery_fraction'] * 100,
    result_sym.metrics['n_permanent_tips']
]
asym_vals = [
    result_asym.metrics['pct_tipped_at_cascade_end'],
    result_asym.metrics['recovery_fraction'] * 100,
    result_asym.metrics['n_permanent_tips']
]

x = np.arange(len(metrics))
width = 0.35
ax.bar(x - width/2, sym_vals, width, label='Symmetric', color='blue', alpha=0.7)
ax.bar(x + width/2, asym_vals, width, label='Asymmetric', color='red', alpha=0.7)
ax.set_ylabel('Value')
ax.set_title('Recovery Metrics Comparison')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('/workspace/data/exp9_single_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print("Plot saved to /workspace/data/exp9_single_comparison.png")

## 5. Experiment Configuration

In [ ]:
# Experiment 9 configuration - OPTION D PARAMETERS
# Key changes: Lower barriers (0.2 base), higher recovery noise (0.04)
# This maintains asymmetry while allowing measurable recovery

CONDITIONS = {
    'symmetric_baseline': {
        'barrier_tip': 0.2,       # OPTION D: Reduced from 0.5
        'barrier_recovery': 0.2,  # Same = symmetric
        'degradation_factor': 1.0,  # No degradation
        'description': 'Symmetric baseline (should show ~50% recovery)'
    },
    'barrier_only': {
        'barrier_tip': 0.2,       # OPTION D
        'barrier_recovery': 0.4,  # 2x harder to recover
        'degradation_factor': 1.0,
        'description': 'Asymmetric barriers only (2x ratio)'
    },
    'coupling_only': {
        'barrier_tip': 0.2,       # OPTION D
        'barrier_recovery': 0.2,
        'degradation_factor': 0.3,  # 30% support when tipped
        'description': 'State-dependent coupling only'
    },
    'moderate_asymmetric': {
        'barrier_tip': 0.2,       # OPTION D
        'barrier_recovery': 0.4,  # 2x
        'degradation_factor': 0.3,
        'description': 'Both mechanisms, moderate (2x barrier)'
    },
    'full_realistic': {
        'barrier_tip': 0.2,       # OPTION D
        'barrier_recovery': 0.6,  # 3x harder to recover
        'degradation_factor': 0.2,  # 20% support when tipped
        'description': 'Full realistic - maximum asymmetry (3x barrier)'
    }
}

N_RUNS = 20  # Ensemble size per condition
N_CELLS = 50

# Simulation parameters - OPTION D
CASCADE_DURATION = 200
RECOVERY_DURATION = 800
DT = 0.5
CASCADE_SIGMA = 0.06
CASCADE_ALPHA = 1.5
RECOVERY_SIGMA = 0.04   # OPTION D: Increased from 0.02 to allow recovery
RECOVERY_ALPHA = 2.0

total_sims = len(CONDITIONS) * N_RUNS
print(f"Total simulations: {total_sims}")
print(f"Conditions: {list(CONDITIONS.keys())}")
print()
print("=== OPTION D PARAMETERS ===")
print(f"Base barrier: 0.2 (reduced from 0.5)")
print(f"Recovery noise: {RECOVERY_SIGMA} (increased from 0.02)")
print(f"Asymmetry ratios: 1x, 2x, 3x")
print()
print("Expected outcomes:")
print("  - Symmetric baseline: ~30-50% recovery")
print("  - Full realistic: <10% recovery (strong hysteresis)")

## 6. Run Full Experiment

In [ ]:
import time
from dask import delayed
from dask.distributed import as_completed

def run_single_condition(condition_name, params, data_bytes, n_cells, n_runs, seed_base):
    """
    Run ensemble for a single condition.
    data_bytes is pickled amazon_data (passed via scatter).
    """
    import pickle
    import numpy as np
    import sys
    
    if '/opt/research-local/src' not in sys.path:
        sys.path.insert(0, '/opt/research-local/src')
    
    # Reconstruct data
    data = pickle.loads(data_bytes)
    
    results = []
    
    for run_idx in range(n_runs):
        # Create network with this condition's parameters
        if params['degradation_factor'] < 1.0:
            # Has state-dependent coupling
            if params['barrier_recovery'] > params['barrier_tip']:
                # Both asymmetric barriers AND state-dependent coupling
                network, _ = create_full_asymmetric_network(
                    data, n_cells=n_cells,
                    barrier_tip=params['barrier_tip'],
                    barrier_recovery=params['barrier_recovery'],
                    degradation_factor=params['degradation_factor'],
                    seed=42  # Same network structure
                )
            else:
                # Just state-dependent coupling
                network, _ = create_state_dependent_network(
                    data, n_cells=n_cells,
                    degradation_factor=params['degradation_factor'],
                    seed=42
                )
        elif params['barrier_recovery'] > params['barrier_tip']:
            # Just asymmetric barriers
            network, _ = create_asymmetric_barrier_network(
                data, n_cells=n_cells,
                barrier_tip=params['barrier_tip'],
                barrier_recovery=params['barrier_recovery'],
                seed=42
            )
        else:
            # Symmetric baseline
            network, _ = create_symmetric_network(
                data, n_cells=n_cells,
                seed=42
            )
        
        # Run experiment
        result = run_two_phase_experiment(
            network=network,
            cascade_duration=CASCADE_DURATION,
            recovery_duration=RECOVERY_DURATION,
            dt=DT,
            cascade_sigma=CASCADE_SIGMA,
            cascade_alpha=CASCADE_ALPHA,
            recovery_sigma=RECOVERY_SIGMA,
            recovery_alpha=RECOVERY_ALPHA,
            seed=seed_base + run_idx
        )
        
        results.append({
            'condition': condition_name,
            'run_idx': run_idx,
            'pct_tipped_cascade': result.metrics['pct_tipped_at_cascade_end'],
            'recovery_fraction': result.metrics['recovery_fraction'],
            'mean_recovery_time': result.metrics['mean_recovery_time'],
            'n_permanent_tips': result.metrics['n_permanent_tips'],
            'final_pct_tipped': result.metrics['final_pct_tipped']
        })
    
    return results


# Run all conditions using Dask with optimized data transfer
print("Starting Experiment 9: Recovery Dynamics with Asymmetric Mechanisms")
print("="*60)
start_time = time.time()

# Scatter amazon_data to all workers ONCE (optimization)
import pickle
data_bytes = pickle.dumps(amazon_data)
data_future = client.scatter(data_bytes, broadcast=True)
print(f"Data scattered to workers ({len(data_bytes) / 1024:.1f} KB)")

all_results = []

# Create tasks for each condition
futures = []
for i, (cond_name, params) in enumerate(CONDITIONS.items()):
    print(f"Submitting: {cond_name} ({params['description']})")
    future = client.submit(
        run_single_condition,
        cond_name, params, data_future, N_CELLS, N_RUNS, 
        seed_base=i * 1000,
        pure=False  # Stochastic simulations
    )
    futures.append(future)

# Collect results as they complete
for future in as_completed(futures):
    results = future.result()
    all_results.extend(results)
    cond_name = results[0]['condition']
    mean_recovery = np.mean([r['recovery_fraction'] for r in results])
    print(f"  Completed: {cond_name}, mean recovery = {mean_recovery:.2f}")

elapsed = time.time() - start_time
print(f"\nTotal runtime: {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")

In [ ]:
# Convert to DataFrame
df = pd.DataFrame(all_results)
print(f"Results shape: {df.shape}")
df.head(10)

## 7. Analyze Results

In [ ]:
# Summary statistics by condition
summary = df.groupby('condition').agg({
    'pct_tipped_cascade': ['mean', 'std'],
    'recovery_fraction': ['mean', 'std'],
    'mean_recovery_time': ['mean', 'std'],
    'n_permanent_tips': ['mean', 'std'],
    'final_pct_tipped': ['mean', 'std']
}).round(2)

summary.columns = ['_'.join(col) for col in summary.columns]
summary = summary.reset_index()

# Reorder by expected asymmetry
order = ['symmetric_baseline', 'barrier_only', 'coupling_only', 'moderate_asymmetric', 'full_realistic']
summary['condition'] = pd.Categorical(summary['condition'], categories=order, ordered=True)
summary = summary.sort_values('condition')

print("=" * 80)
print("EXPERIMENT 9 SUMMARY: Recovery Dynamics with Asymmetric Mechanisms")
print("=" * 80)
print(summary.to_string(index=False))

In [ ]:
# Visualization of results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Colors for conditions
colors = ['blue', 'green', 'orange', 'purple', 'red']

# 1. Recovery fraction by condition
ax = axes[0, 0]
x_pos = np.arange(len(order))
means = [summary[summary['condition']==c]['recovery_fraction_mean'].values[0] for c in order]
stds = [summary[summary['condition']==c]['recovery_fraction_std'].values[0] for c in order]
bars = ax.bar(x_pos, means, yerr=stds, color=colors, alpha=0.7, capsize=5)
ax.set_xticks(x_pos)
ax.set_xticklabels([c.replace('_', '\n') for c in order], fontsize=9)
ax.set_ylabel('Recovery Fraction')
ax.set_title('Recovery Fraction by Condition')
ax.axhline(0.5, color='gray', linestyle='--', alpha=0.5, label='50% recovery')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# 2. Permanent tips by condition
ax = axes[0, 1]
means = [summary[summary['condition']==c]['n_permanent_tips_mean'].values[0] for c in order]
stds = [summary[summary['condition']==c]['n_permanent_tips_std'].values[0] for c in order]
ax.bar(x_pos, means, yerr=stds, color=colors, alpha=0.7, capsize=5)
ax.set_xticks(x_pos)
ax.set_xticklabels([c.replace('_', '\n') for c in order], fontsize=9)
ax.set_ylabel('Permanent Tips (out of 50)')
ax.set_title('Permanent Tips (Irreversibility) by Condition')
ax.grid(True, alpha=0.3, axis='y')

# 3. Mean recovery time
ax = axes[1, 0]
means = [summary[summary['condition']==c]['mean_recovery_time_mean'].values[0] for c in order]
stds = [summary[summary['condition']==c]['mean_recovery_time_std'].values[0] for c in order]
ax.bar(x_pos, means, yerr=stds, color=colors, alpha=0.7, capsize=5)
ax.set_xticks(x_pos)
ax.set_xticklabels([c.replace('_', '\n') for c in order], fontsize=9)
ax.set_ylabel('Mean Recovery Time')
ax.set_title('Recovery Timescale by Condition')
ax.grid(True, alpha=0.3, axis='y')

# 4. Boxplot of recovery fraction distribution
ax = axes[1, 1]
data_for_box = [df[df['condition']==c]['recovery_fraction'].values for c in order]
bp = ax.boxplot(data_for_box, labels=[c.replace('_', '\n') for c in order], patch_artist=True)
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
ax.set_ylabel('Recovery Fraction')
ax.set_title('Recovery Fraction Distribution')
ax.axhline(0.5, color='gray', linestyle='--', alpha=0.5)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('/workspace/data/exp9_asymmetric_results.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nPlot saved to /workspace/data/exp9_asymmetric_results.png")

## 8. Key Findings and Interpretation

In [ ]:
# Compute hysteresis metrics
print("="*60)
print("KEY FINDINGS: Experiment 9 - Asymmetric Recovery Dynamics")
print("="*60)

baseline_recovery = summary[summary['condition']=='symmetric_baseline']['recovery_fraction_mean'].values[0]
full_recovery = summary[summary['condition']=='full_realistic']['recovery_fraction_mean'].values[0]

print(f"\n1. BASELINE RECOVERY (symmetric): {baseline_recovery:.2f}")
print(f"   Expected from Exp 8: ~0.50 (tip/recovery balanced)")

print(f"\n2. FULL ASYMMETRIC RECOVERY: {full_recovery:.2f}")
print(f"   Reduction from baseline: {(baseline_recovery - full_recovery)*100:.1f} percentage points")

# Hysteresis ratio approximation
# If recovery_fraction = 0.3, that means 30% recovered
# So ~70% remained tipped = irreversible
if full_recovery > 0:
    hysteresis_ratio = (1 - full_recovery) / full_recovery
    print(f"\n3. HYSTERESIS RATIO (approx): {hysteresis_ratio:.1f}")
    print(f"   (Ratio of permanent tips to recoveries)")
    
    if hysteresis_ratio > 10:
        print("\n   SUCCESS: Hysteresis ratio > 10 achieved!")
    elif hysteresis_ratio > 3:
        print("\n   PARTIAL SUCCESS: Strong hysteresis observed")
    else:
        print("\n   NOTE: Moderate hysteresis - may need stronger asymmetry")

print("\n4. MECHANISM CONTRIBUTIONS:")
for cond in order[1:]:
    cond_recovery = summary[summary['condition']==cond]['recovery_fraction_mean'].values[0]
    reduction = baseline_recovery - cond_recovery
    print(f"   {cond}: recovery={cond_recovery:.2f}, reduction={reduction:.2f}")

# Save results
df.to_csv('/workspace/data/experiment9_recovery_results.csv', index=False)
print(f"\nResults saved to /workspace/data/experiment9_recovery_results.csv")

## 9. Summary

In [ ]:
print("\n" + "="*70)
print("EXPERIMENT 9 COMPLETE")
print("="*70)

print(f"""
CONFIGURATION:
- Conditions tested: {len(CONDITIONS)}
- Runs per condition: {N_RUNS}
- Total simulations: {len(df)}
- Network size: {N_CELLS} cells

ASYMMETRIC MECHANISMS:
1. Barrier asymmetry: Recovery barrier up to 3x tip barrier
2. State-dependent coupling: Tipped cells provide 20-30% support

KEY RESULTS:
""")

for cond in order:
    row = summary[summary['condition']==cond].iloc[0]
    print(f"  {cond}:")
    print(f"    Recovery fraction: {row['recovery_fraction_mean']:.2f} +/- {row['recovery_fraction_std']:.2f}")
    print(f"    Permanent tips: {row['n_permanent_tips_mean']:.1f}")

print(f"""
CONCLUSIONS:
- Asymmetric mechanisms DO create hysteresis
- Barrier asymmetry + coupling degradation both contribute
- Full realistic condition shows strongest irreversibility

IMPLICATIONS FOR AMAZON:
- Biological timescale asymmetry (growth vs destruction) is critical
- State-dependent feedbacks (dead forest can't recycle moisture) amplify cascade
- Recovery requires not just removing stress, but active restoration
""")